In [10]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema import Document
os.environ["OPENAI_API_KEY"] = "..."

openai_embedding=OpenAIEmbeddings(model = 'text-embedding-3-small')

In [2]:
def load_and_split_pdf(file_path):
    loader = PyPDFLoader(file_path)
    return loader.load_and_split()

In [3]:
def create_vector_store(_docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(_docs)
    persist_directory = "./chroma_db"
    vectorstore = Chroma.from_documents(
        split_docs, 
        OpenAIEmbeddings(model='text-embedding-3-small'),
        persist_directory=persist_directory
    )
    return vectorstore

In [4]:
def add_to_vector_store(new_docs, persist_directory="./chroma_db"):
    # 1. 기존 벡터 저장소 불러오기
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
    )

    # 2. 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(new_docs)

    # 3. 문서 추가
    vectorstore.add_documents(split_docs)

    return vectorstore

In [5]:
pdf_dir = r"C:\python workspace\3-3\챗봇\논문"
pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith(".pdf")]

In [6]:
for pdf in pdf_files:
    path = os.path.join(pdf_dir, pdf)
    sp = load_and_split_pdf(path)
    add_to_vector_store(sp)

C:\Users\mansr\AppData\Local\Temp\ipykernel_12092\3363452400.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [8]:
# load from disk
db3 = Chroma(
    persist_directory="./chroma_db",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
)

query = "junhwa chi's paper?"
result = db3.similarity_search(query)
print(result[0].page_content)

IEEE Trans. Geosci. Remote Sens., vol. 46, no. 12, pp. 4173–4185, Dec.
2008.
[48] X. Huang, L. Zhang, and L. Wang,“Evaluation of morphological texture
features for mangrove forest mapping and species discrimination using
multispectral IKONOS imagery,” IEEE Geosci. Remote Sens. Lett., vol. 6,
no. 3, pp. 393–397, Jul. 2009.
Junhwa Chi(S’10–M’14) received the B.S. degree in
geoinformatic engineering, the second B.S. degree in
computer science from Inha University, Incheon,
SouthKorea,in2006,theM.S.degreeingeoinformatic
engineeringin 2008,and thePh.D.degree in geomatics
engineering from the School of Civil Engineering,
Purdue University, West Lafayette, IN, USA, in 2013.
Currently, he works as a Postdoctoral Scholar with
the Center for Spatial Technologies and Remote Sens-
ing (CSTARS), Department of Land, Air, and Water
Resources, University of California, Davis, CA, USA.
His research interests include agricultural/environmental remote sensing applica-


In [9]:
db3 = Chroma(
    persist_directory="./chroma_db",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
)

query = "explain hyungbin moon's 2020 paper"
docs = db3.similarity_search(query, k=15)

formatted_docs = [Document(page_content=doc.page_content) for doc in docs]

prompt_template = """
다음은 논문 일부입니다. 이 내용을 바탕으로 석사 논문 내용을 연구대상 및 방법, 연구방법, 연구결과, 제언을 핵심적으로 요약해 주세요.
논문 내용:
{context}
요약:
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context"])
llm = ChatOpenAI(model="gpt-4o")
chain = LLMChain(llm=llm, prompt=PROMPT)
context_text = "\n\n".join([doc.page_content for doc in formatted_docs])
summary = chain.run(context=context_text)
print(summary, "\n")

C:\Users\mansr\AppData\Local\Temp\ipykernel_12092\2887330060.py:19: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o")
C:\Users\mansr\AppData\Local\Temp\ipykernel_12092\2887330060.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=PROMPT)
C:\Users\mansr\AppData\Local\Temp\ipykernel_12092\2887330060.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = chain.run(context=context_text)


**연구대상 및 방법:**
이 연구는 한국에서의 사용후핵연료 관리 정책에 대한 대중의 인식과 선호도를 분석하기 위한 목적으로 수행되었습니다. 연구 데이터는 2020년 1월에 전문 조사 기관인 Macromil Embrain을 통해 온라인 설문조사를 통해 수집되었으며, 설문조사 참여자는 성별, 연령, 지역을 고려한 비례층화 표본추출 방법을 통해 선정된 20세에서 59세 사이의 성인 1,000명이었습니다.

**연구방법:**
연구는 사용후핵연료 관리에 대한 대중의 태도와 선호를 개인 및 세그먼트 수준에서 분석하기 위해 설문조사 데이터를 활용한 통계적 분석을 진행하였습니다. 분석에는 다변량 통계 기법이 적용되어 정책에 대한 대중의 다양한 반응과 인식을 구체적으로 파악하고자 하였습니다.

**연구결과:**
연구 결과, 대중은 사용후핵연료 저장소의 위험성과 관련하여 부정적인 결과들에 대한 우려를 표명하였으며, 전반적으로 정책에 대한 수용도는 시간이 지남에 따라 감소하는 경향을 보였습니다. 또한, 사용후핵연료 관리에 대한 대중의 인식은 지리적 요인과 밀접하게 관련되어 있으며, 지역별 기상 조건을 고려하여 개별 정책이 마련되고 시행되어야 한다는 점이 강조되었습니다.

**제언:**
연구는 사용후핵연료 관리 정책의 사회적 합의를 위해서는 대중의 인식과 선호를 반영한 정책 마련이 필요하다고 제언합니다. 또한, 지역적 기상 조건을 고려하여 맞춤형 정책을 수립하고, 정책 결정 과정에서 대중과 전문가 간의 소통을 강화할 필요가 있습니다. 이를 통해 보다 효과적이고 수용 가능한 사용후핵연료 관리 정책을 구현할 수 있을 것입니다. 

